In [187]:
import import_asl
import importlib
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import re
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features
from sklearn.metrics import precision_recall_fscore_support
from tsfresh.feature_extraction import MinimalFeatureExtractionSettings, ReasonableFeatureExtractionSettings
from tsfresh.feature_extraction import FeatureExtractionSettings
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from scipy.stats import mode

In [38]:
importlib.reload(import_asl)

<module 'import_asl' from 'C:\\Users\\anura\\Documents\\Machine-Learning-Course-Project\\import_asl.py'>

In [45]:
def grid_search_length(length):   
    signs_all_ts = import_asl.import_data(length)
    signs_flat = import_asl.create_flat_df(signs_all_ts, spatial_scale=True)

    X = signs_flat.drop(['label', 'ID', 'folder'], axis=1)

    Y = signs_flat['label']
    le = LabelEncoder()
    le.fit(Y)
    y = le.transform(Y)

    clf = LogisticRegression(C=1, penalty='l1')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    print(cross_val_score(clf, X_train, y_train, verbose=1).mean())

In [ ]:
for length in [10, 20, 30, 40, 57]:
    grid_search_length(length)

## Baseline Model

In [51]:
signs_all_ts = import_asl.import_data(57)
signs_flat_57 = import_asl.create_flat_df(signs_all_ts, spatial_scale=False)
X = signs_flat_57.drop(['label', 'ID', 'folder'], axis=1)

Y = signs_flat_57['label']
le = LabelEncoder()
le.fit(Y)
y = le.transform(Y)

clf = svm.SVC(kernel='linear', C=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(cross_val_score(clf, X_train, y_train, verbose=1).mean())
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.1s finished


0.90752624199


### Test Accuracy

In [57]:
sum(clf.predict(X_test) == y_test)/len(y_test)

0.93146417445482865

In [56]:
precision_recall_fscore_support(y_test, clf.predict(X_test), average='macro')

(0.93884148015726954, 0.93718348902559445, 0.93155490328294632, None)

## SVM with spatial scaling and downsampling

In [58]:
signs_all_ts = import_asl.import_data(10)
signs_flat = import_asl.create_flat_df(signs_all_ts, spatial_scale=True)
X = signs_flat.drop(['label', 'ID', 'folder'], axis=1)

Y = signs_flat['label']
le = LabelEncoder()
le.fit(Y)
y = le.transform(Y)

clf = svm.SVC(kernel='linear', C=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(cross_val_score(clf, X_train, y_train, verbose=1).mean())

tctodd1
tctodd2
tctodd3
tctodd4
tctodd5
tctodd6
tctodd7
tctodd8
tctodd9


C:\Users\anura\Documents\Machine-Learning-Course-Project\import_asl.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['frame'] = df.index + 1
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.6s finished


0.952723215676


## Right Hand Meta Features

In [61]:
df = signs_all_ts
df2 = signs_flat
df.ID = df.ID.astype(str)
df['ID'] = df['folder'] + df['label'] + df['ID']

g = df.groupby('ID')
df['ix'] = df.index.values

df['frame'] = g['ix'].rank(method='first')

df['frame'] = df['frame'].astype(int)

#y = df[['ID','label']].drop_duplicates()['label']
y = df[['ID','label']].drop_duplicates().set_index('ID')
df.drop(['label', 'folder', 'ix'], axis = 1, inplace=True)

df_R = df.ix[:,11:]


In [97]:
settings = FeatureExtractionSettings()

extracted_features = extract_features(df_R, 
                                      column_id="ID", 
                                      column_sort="frame",
                                     feature_extraction_settings=ReasonableFeatureExtractionSettings())

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 11/11 [11:22<00:00, 62.01s/it]


In [98]:
for col in extracted_features.columns:
    if (sum(pd.isnull(extracted_features[col])) > 0) or (sum(extracted_features[col]) == 0):
        del (extracted_features[col])

In [169]:
#extracted_features.columns
candidates = ['mean_autocorrelation', 'absolute_sum_of_changes', 'large_number_of_peaks', 'skewness', 
              'thumb_R_symmetry_looking','fore_R_symmetry_looking','little_R_symmetry_looking', 
              'mean_second_derivate_central', 'last_location_of_maximum',
             ]
filtered_features = list(extract_features(df_R, 
                                      column_id="ID", 
                                      column_sort="frame",
                                     feature_extraction_settings=MinimalFeatureExtractionSettings()).columns.values)
for col in extracted_features.columns:
    for candidate in candidates:
        if col.find(candidate) > -1:
            filtered_features.append(col)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 11/11 [00:10<00:00,  1.05it/s]


In [171]:
for col in filtered_features:
    if col.find('R__length') > -1:
        filtered_features.remove(col)

In [172]:
y = y.sort_index()
X = np.array(extracted_features[filtered_features].sort_index())
X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size=0.25, random_state=42)
clf = RandomForestClassifier(n_estimators=50,  criterion='entropy')
clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [138]:
param_grid = {"max_depth": [10, 30, None],
              "max_features": [20, 50, 'auto'],
              "min_samples_split": [2, 5],
              "min_samples_leaf": [1, 3],
              "n_estimators": [10, 20, 50],
              #"bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

grid_search = GridSearchCV(clf, param_grid=param_grid, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10 
[CV]  min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10, total=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10 
[CV]  min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10, total=   0.2s
[CV] min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10 
[CV]  min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=10, total=   0.2s
[CV] min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=20 
[CV]  min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=20, total=   0.5s
[CV] min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=20 
[CV]  min_samples_split=2, criterion=gini, max_depth=10, max_features=20, min_samples_leaf=1, n_estimators=20, total=   0.5s
[CV] min_samples_split=2, criterion=gini, max_depth=10, max_

KeyboardInterrupt: 

In [105]:
grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [174]:
cross_val_score(clf, X_train, y_train, cv=5).mean()

0.94853212488575234

In [173]:
extracted_features[filtered_features].columns[clf.feature_importances_ < 0.001]

Index(['pitch_R__minimum', 'little_R__median', 'little_R__minimum',
       'y_R__median', 'y_R__minimum', 'middle_R__minimum', 'middle_R__median',
       'x_R__median', 'z_R__mean', 'z_R__median', 'z_R__sum_values',
       'roll_R__median', 'thumb_R__minimum', 'thumb_R__median',
       'ring_R__last_location_of_maximum',
       'thumb_R__last_location_of_maximum'],
      dtype='object')

In [176]:
sum(clf.predict(X_test) == y_test)/len(y_test)

0.95015576323987538

### Ensemble: Voting

In [180]:
df2['ID'] = df2['folder'] + df2['label'] + df2['ID'].astype(str)

In [182]:
X1 = df2.set_index('ID').drop(['folder', 'label'], axis=1)

In [186]:
X2 = extracted_features[filtered_features]
X = X1.merge(X2, right_index=True, left_index=True)

In [ ]:
clf1 = LogisticRegression(C=1, penalty='l1')
clf2 = svm.SVC(kernel='linear',C=1,probability=True)
clf3 = RandomForestClassifier(n_estimators=50,  criterion='entropy')
clf4 = KNeighborsClassifier(n_neighbors=1)

clf1.fit(X1, y)
clf2.fit(X1, y)
clf3.fit(X2, y)
clf4.fit(X1, y)



In [ ]:
def eclf_predict(X, y, clf1, clf2, clf3, clf4):
    X1 = X.ix[:,:220]
    X2 = X.ix[:,220:]
        
    y_pred1 = clf1.predict(X1)
    y_pred2 = clf2.predict(X1)
    y_pred3 = clf3.predict(X2)
    y_pred4 = clf4.predict(X1)
    
    for i in range(len(y)):
        y_pred = mode([y_pred1[i], y_pred2[i], y_pred3[i], y_pred4[i]])
        
    